In [ ]:
import twint
import nest_asyncio
import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

nest_asyncio.apply()

### Before

In [ ]:
c = twint.Config()
c.Username = 'Prambors'
c.Since = "2020-08-01" #batas awal tanggal pengambilan tweet
c.Until = "2021-01-31" #batas akhir tanggal pengambilan tweet
c.Search = "covid OR vaksin"
c.Count = True         
c.Retweets = True
c.Stats = True 
c.Show_hashtags = True
c.Hide_output = True
c.Profile_full = True
c.Store_json = True
c.Output = './prambors/before.json' # save to json
twint.run.Search(c)

### After

In [ ]:
c = twint.Config()
c.Username = 'Prambors'
c.Since = "2021-02-01" #batas awal tanggal pengambilan tweet
c.Search = "covid OR vaksin"
c.Count = True         
c.Retweets = True
c.Stats = True 
c.Show_hashtags = True
c.Hide_output = True
c.Profile_full = True
c.Store_json = True
c.Output = './prambors/after.json' # save to json
twint.run.Search(c)

In [ ]:
df_before = pd.read_json('./prambors/before.json' , lines = True)
df_after = pd.read_json('./prambors/after.json' , lines = True)

In [ ]:
# df_before.duplicated()
df_before = df_before[['id', 'conversation_id', 'user_id', 'username', 'name','tweet','replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url','user_rt_id', 'user_rt', 'retweet_id', 'reply_to']]

df_before = df_before.drop_duplicates(subset='link')
df_before

In [ ]:
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
# from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
import numpy as np
from datetime import datetime, timedelta

In [ ]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
# chrome_options.add_argument('headless') # untuk menutup browser

#specify the path to chromedriver.exe
driver = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)

In [ ]:
driver.get('https://www.twitter.com/login')
sleep(2)

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('dyingrabbit_') # Ubah username sesuai akun pribadi

my_password = getpass()

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

In [ ]:
data_tweets_before = df_before

In [ ]:
def get_list_replies(link):
    list_replies = []
    driver.get(link)
    sleep(2)
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)
    
    try:
        replies_container = driver.find_element_by_xpath('//div[@aria-label="Timeline: Conversation"]')
        if replies_container:
            replies = replies_container.find_elements_by_xpath('.//div[@dir="ltr"]/span')
            for reply in replies:
                list_replies.append(reply.text)
    except:
        print(list_replies)
        return list_replies
    print(list_replies)
    return list_replies

In [ ]:
arr_temp = []
for i, row in data_tweets_before.iterrows():
    print(i)
    arr_temp.append(get_list_replies(row['link']))

data_tweets_before['list_replies'] = arr_temp

In [ ]:
data_tweets_before.to_json('./prambors/before_replies.json')

In [ ]:
data_tweets_before = pd.read_json('./prambors/before_replies.json')
data_tweets_before

In [ ]:
arr = []
for i in data_tweets_before['list_replies']:
    for x in i:
        arr.append(x)
arr = list(set(arr))

prefix = '@'
arr = [i for i in arr if prefix in i]
arr = list(set(arr))
arr.remove('@Prambors')
arr.remove('@___bacot')
print(len(arr))
arr

In [ ]:
filename = './prambors/before_user_data.json'
i = 1
for uname in arr:
    uname = uname[1:]
    print(i, uname)
    i = i+1
    c = twint.Config()
    c.Username = uname
    c.Since = "2020-08-01" #batas awal tanggal pengambilan tweet
    c.Until = "2021-01-31" #batas akhir tanggal pengambilan tweet
    c.Search = "covid OR vaksin"
    c.Count = True         
    c.Retweets = True
    c.Stats = True 
    c.Show_hashtags = True
    c.Hide_output = True
    c.Profile_full = True
    c.Store_json = True
    c.Output = filename # save to json
    twint.run.Search(c)

In [ ]:
before_user_data_tweets = pd.read_json('./prambors/before_user_data.json', lines = True)
before_user_data_tweets = before_user_data_tweets[['id', 'conversation_id', 'user_id', 'username', 'name','tweet','replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url','user_rt_id', 'user_rt', 'retweet_id', 'reply_to']]

before_user_data_tweets = before_user_data_tweets.drop_duplicates(subset='link')
for i in before_user_data_tweets['link']:
    print(i)

In [ ]:
arr_temp = []
for i, row in before_user_data_tweets.iterrows():
     arr_temp.append(get_list_replies(row['link']))

before_user_data_tweets['list_replies'] = arr_temp
before_user_data_tweets['list_replies']

In [ ]:
before_user_data_tweets.to_json('./data/before_user_scrap.json')

### SCRAPPING AFTER DATA

In [ ]:
df_after = pd.read_json('./prambors/after.json' , lines = True)
df_after = df_after[['id', 'conversation_id', 'user_id', 'username', 'name','tweet','replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url','user_rt_id', 'user_rt', 'retweet_id', 'reply_to']]

df_after = df_after.drop_duplicates(subset='link')
for i in df_after['link']:
    print(i)

In [ ]:
data_tweets_after = df_after

In [ ]:
arr_temp = []
for i, row in data_tweets_after.iterrows():
     arr_temp.append(get_list_replies(row['link']))

data_tweets_after['list_replies'] = arr_temp
data_tweets_after['list_replies']

In [ ]:
data_tweets_after.to_json('./prambors/after_replies.json')

In [ ]:
data_tweets_after = pd.read_json('./prambors/after_replies.json')
data_tweets_after

In [ ]:
arr = []
for i in data_tweets_after['list_replies']:
    for x in i:
        arr.append(x)
arr = list(set(arr))

prefix = '@'
arr = [i for i in arr if prefix in i]
arr = list(set(arr))
arr.remove('@Prambors')
print(len(arr))
arr

In [ ]:
filename = './prambors/after_user_data.json'
for uname in arr:
    uname = uname[1:]
    c = twint.Config()
    c.Username = uname
    c.Since = "2021-02-01" #batas awal tanggal pengambilan tweet
#     c.Until = "2021-01-31" #batas akhir tanggal pengambilan tweet
    c.Search = "covid OR vaksin"
    c.Count = True         
    c.Retweets = True
    c.Stats = True 
    c.Show_hashtags = True
    c.Hide_output = True
    c.Profile_full = True
    c.Store_json = True
    c.Output = filename # save to json
    twint.run.Search(c)

In [ ]:
after_user_data_tweets = pd.read_json('./prambors/after_user_data.json', lines = True)
after_user_data_tweets = after_user_data_tweets[['id', 'conversation_id', 'user_id', 'username', 'name','tweet','replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url','user_rt_id', 'user_rt', 'retweet_id', 'reply_to']]

after_user_data_tweets = after_user_data_tweets.drop_duplicates(subset='link')
for i in after_user_data_tweets['link']:
    print(i)

In [ ]:
arr_temp = []
for i, row in after_user_data_tweets.iterrows():
     arr_temp.append(get_list_replies(row['link']))

after_user_data_tweets['list_replies'] = arr_temp
after_user_data_tweets['list_replies']

In [ ]:
after_user_data_tweets.to_json('./prambors/after_user_scrap.json')

In [ ]:
after_user_data_tweets = pd.read_json('./prambors/after_user_scrap.json')
after_user_data_tweets.columns

In [ ]:
arr = []
for i in after_user_data_tweets['list_replies']:
    for x in i:
        arr.append(x)
arr = list(set(arr))

prefix = '@'
arr = [i for i in arr if prefix in i]
arr = list(set(arr))
print(len(arr))
arr